# IVI NLP case

In [463]:
# Imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import pymorphy2

## I. Pars news for the ~half a year 2019
We'll get news from 1.01.2019 to 6.05.2019

In [271]:
# Create dataframe
kinopoisk = pd.DataFrame(columns=['id', 'from_title', 'from_text'])

# Get html's id
ind = [i for i in range(3308146, 3376082)]

for i in range(len(ind)):
    url = requests.get('https://www.kinopoisk.ru/media/news/{}/'.format(ind[i]))

    # Create soup object
    # Check for response
    if url.status_code == 200:
        soup = BeautifulSoup(url.text)
        kinopoisk.loc[i, 'id'] = ind[i]
    
        # Form regexp to serch names in quotes
        pattern = r'«([\S\s]+?)»'
    
        # Get films from title
        title = str(soup.title)
        kinopoisk.loc[i, 'from_title'] = re.findall(pattern, title)
    
        # Get text
        text = str(soup.find(class_="media-post-body media-post-body_theme_desktop").find_all('p'))
        kinopoisk.loc[i, 'from_text'] = re.findall(pattern, text)

kinopoisk

,id,from_title,from_text
3914,3312060,"[Ведьмак, Щегол, Тобол]","[Обещание на рассвете, Сезар, Обещание на расс..."
3915,3312061,[Бархатной бензопилы],"[Стрингера, Роман Израэл, Esq., Бархатной бенз..."
4158,3312304,[],"[Охота, Прочь, Охоте, Однажды в Голливуде]"
4160,3312306,"[Оскара, Мстителей]","[Оскар, Золотых глобусах, Мстителей, Оскара, М..."
4180,3312326,[],[Войны бесконечности]
...,...,...,...
67932,3376078,[Джессики Джонс],"[У каждого есть своя история, — говорит героин..."
67933,3376079,[Выше неба],"[Кинотавра, Выше неба, Хороший мальчик, Выше н..."
67934,3376080,[Гроза],"[Почему люди не летают, как птицы, сударь, Бит..."
67935,3376081,[Сторож],"[Судите строго!, Завод, Сторожа, Я хотел сдела..."


## II. Work with dataframe
We'll drop excess information and normalize word form for each title 

In [483]:
morph = pymorphy2.MorphAnalyzer()

#__Normalize form from list
def morf_lem_list(texts):
    texts_out = []
    for i in texts:
        new = i.split(' ')
        sent = [morph.parse(n)[0].normal_form for n in new]
        outp = (' ').join(sent)
        texts_out.append(outp)
    return texts_out

In [484]:
kinopoisk = kinopoisk.reset_index(drop=True)
# Keep all titles in one columns
kinopoisk.loc[:, 'all_titles'] =  kinopoisk.loc[:, 'from_title'] + kinopoisk.loc[:, 'from_text']
# Normalize form
kinopoisk.loc[:, 'normal_title'] = kinopoisk.loc[:, 'all_titles'].apply(morf_lem_list)
kinopoisk.head()

,id,from_title,from_text,all_titles,normal_title
0,3312060,"[Ведьмак, Щегол, Тобол]","[Обещание на рассвете, Сезар, Обещание на расс...","[Ведьмак, Щегол, Тобол, Обещание на рассвете, ...","[ведьмак, щегол, тобол, обещание на рассвет, с..."
1,3312061,[Бархатной бензопилы],"[Стрингера, Роман Израэл, Esq., Бархатной бенз...","[Бархатной бензопилы, Стрингера, Роман Израэл,...","[бархатный бензопила, стрингер, роман израэл, ..."
2,3312304,[],"[Охота, Прочь, Охоте, Однажды в Голливуде]","[Охота, Прочь, Охоте, Однажды в Голливуде]","[охота, прочь, охота, однажды в голливуд]"
3,3312306,"[Оскара, Мстителей]","[Оскар, Золотых глобусах, Мстителей, Оскара, М...","[Оскара, Мстителей, Оскар, Золотых глобусах, М...","[оскар, мститель, оскар, золотой глобус, мстит..."
4,3312326,[],[Войны бесконечности],[Войны бесконечности],[война бесконечность]


In [476]:
#__Normalize form from string
def morf_lem_string(text):
    text = text.split(' ')
    texts_out = []
    for i in text:
        w = morph.parse(i)[0].normal_form
        texts_out.append(w)
    return (' ').join(texts_out)

In [477]:
# Import DF with titles
films = pd.read_csv('films.csv', sep=';')
films = films.loc[:, ['title']]
films.loc[:, 'normal_title'] = films.loc[:, 'title'].apply(morf_lem_string)
films.head()

,title,normal_title
0,За деревьями,за дерево
1,Сиротский Бруклин,сиротский бруклин
2,Тайная жизнь домашних животных 2,тайный жизнь домашний животное 2
3,Борьба с моей семьей,борьба с мой семья
4,BH90210,bh90210


## III. Count occurrences of titles in news
We'll work with normalized titles, than return them in the original format

In [478]:
#__Count occurrences of films in news
def dictionary_count(films_data, news_data):
    d = {}
    for i in range (len(news_data)):
        for n in range (len(films_data)):
            num = news_data.loc[i, 'normal_title'].count(films_data.loc[n, 'normal_title'])
            if num > 0:
                d[news_data.loc[i, 'id']] = (films_data.loc[n, 'title'], num)
    return d
    

In [482]:
# Get dictionart
title_newid_count = dictionary_count(films, kinopoisk)
title_newid_count

{3312060: ('Капитан Марвел', 1),
 3312306: ('Мстители: Финал', 1),
 3312878: ('Алита: Боевой ангел', 1),
 3313121: ('Другой', 1),
 3313678: ('Игра престолов', 3),
 3313696: ('Мстители: Финал', 1),
 3313709: ('Стекло', 1),
 3313827: ('Пиноккио', 5),
 3313956: ('Игра престолов', 4),
 3314071: ('Игра престолов', 1),
 3314096: ('Т-34', 4),
 3314252: ('Человек-Паук: Вдали от дома', 2),
 3314258: ('Стекло', 8),
 3314471: ('Игра престолов', 1),
 3314481: ('Человек-Паук: Вдали от дома', 1),
 3314667: ('Капитан Марвел', 1),
 3314783: ('Игра престолов', 3),
 3314819: ('Стекло', 2),
 3315132: ('Море соблазна', 1),
 3315185: ('Юморист', 3),
 3315200: ('Шазам!', 2),
 3315215: ('Джон Уик 3', 1),
 3316002: ('Тайная жизнь домашних животных 2', 1),
 3316021: ('Стекло', 1),
 3316593: ('Оно 2', 2),
 3317095: ('Стекло', 8),
 3317212: ('Мстители: Финал', 2),
 3317244: ('Король Лев', 1),
 3317437: ('Т-34', 1),
 3317514: ('Игра престолов', 1),
 3317741: ('Джон Уик 3', 1),
 3318260: ('Лев Яшин. Вратарь моей м

In [481]:
# Create dataframe easy to work with
counted_titles = pd.DataFrame.from_dict(title_newid_count, orient='index').reset_index()
counted_titles.columns = ['id', 'title', 'count_v']
counted_titles

,id,title,count_v
0,3312060,Капитан Марвел,1
1,3312306,Мстители: Финал,1
2,3312878,Алита: Боевой ангел,1
3,3313121,Другой,1
4,3313678,Игра престолов,3
...,...,...,...
500,3375539,Мидуэй,1
501,3375542,Игра престолов,1
502,3375632,Игра престолов,1
503,3375667,Игра престолов,3
